# Recurrent Neural Network.
#A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
#This example is using the MNIST database of handwritten digits 

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
learning_rate = 0.001
training_steps = 20000
batch_size = 128
display_step = 1

In [4]:
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

In [5]:
A = tf.placeholder("float", [None, timesteps, num_input])
B = tf.placeholder("float", [None, num_classes])

In [6]:
print(A)

Tensor("Placeholder:0", shape=(?, 28, 28), dtype=float32)


In [7]:
print(B)

Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)


In [8]:
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(A, weights, biases)
prediction = tf.nn.softmax(logits)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=B))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(B, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={A: batch_x, B: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={A: batch_x,
                                                                 B: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={A: test_data, B: test_label}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step 1, Minibatch Loss= 2.7028, Training Accuracy= 0.023
Step 2, Minibatch Loss= 2.5222, Training Accuracy= 0.023
Step 3, Minibatch Loss= 2.8960, Training Accuracy= 0.016
Step 4, Minibatch Loss= 2.7504, Training Accuracy= 0.016
Step 5, Minibatch Loss= 2.6792, Training Accuracy= 0.047
Step 6, Minibatch Loss= 2.6940, Training Accuracy= 0.047
Step 7, Minibatch Loss= 2.8098, Training Accuracy= 0.016
Step 8, Minibatch Loss= 2.6053, Training Accuracy= 0.047
Step 9, Minibatch Loss= 2.6555, Training Accuracy= 0.039
Step 10, Minibatch Loss= 2.7475, Training Accuracy= 0.039
Step 11, Minibatch Loss= 2.6062, Training Accuracy= 0.062
Step 12, Minibatch Loss= 2.5663, Training Accuracy= 0.039
Step 13, Minibatch Loss= 2.5607, Training Accuracy= 0.047
Step 14, Minibatch Loss= 2.5354, Training Accuracy= 0.086
Step 1

Step 137, Minibatch Loss= 2.2016, Training Accuracy= 0.188
Step 138, Minibatch Loss= 2.1362, Training Accuracy= 0.242
Step 139, Minibatch Loss= 2.2095, Training Accuracy= 0.188
Step 140, Minibatch Loss= 2.1646, Training Accuracy= 0.211
Step 141, Minibatch Loss= 2.2359, Training Accuracy= 0.164
Step 142, Minibatch Loss= 2.1651, Training Accuracy= 0.281
Step 143, Minibatch Loss= 2.1640, Training Accuracy= 0.234
Step 144, Minibatch Loss= 2.1828, Training Accuracy= 0.211
Step 145, Minibatch Loss= 2.1309, Training Accuracy= 0.305
Step 146, Minibatch Loss= 2.1499, Training Accuracy= 0.266
Step 147, Minibatch Loss= 2.1099, Training Accuracy= 0.289
Step 148, Minibatch Loss= 2.1404, Training Accuracy= 0.250
Step 149, Minibatch Loss= 2.1795, Training Accuracy= 0.258
Step 150, Minibatch Loss= 2.1545, Training Accuracy= 0.234
Step 151, Minibatch Loss= 2.2612, Training Accuracy= 0.172
Step 152, Minibatch Loss= 2.1427, Training Accuracy= 0.227
Step 153, Minibatch Loss= 2.1909, Training Accuracy= 0.2

Step 277, Minibatch Loss= 2.0698, Training Accuracy= 0.289
Step 278, Minibatch Loss= 2.0864, Training Accuracy= 0.250
Step 279, Minibatch Loss= 2.0263, Training Accuracy= 0.305
Step 280, Minibatch Loss= 2.0083, Training Accuracy= 0.328
Step 281, Minibatch Loss= 2.0285, Training Accuracy= 0.312
Step 282, Minibatch Loss= 1.9983, Training Accuracy= 0.367
Step 283, Minibatch Loss= 2.0619, Training Accuracy= 0.336
Step 284, Minibatch Loss= 2.0217, Training Accuracy= 0.312
Step 285, Minibatch Loss= 1.9604, Training Accuracy= 0.383
Step 286, Minibatch Loss= 2.0345, Training Accuracy= 0.352
Step 287, Minibatch Loss= 2.0716, Training Accuracy= 0.375
Step 288, Minibatch Loss= 2.0067, Training Accuracy= 0.297
Step 289, Minibatch Loss= 1.9872, Training Accuracy= 0.383
Step 290, Minibatch Loss= 2.0023, Training Accuracy= 0.328
Step 291, Minibatch Loss= 2.0534, Training Accuracy= 0.289
Step 292, Minibatch Loss= 2.0473, Training Accuracy= 0.336
Step 293, Minibatch Loss= 2.0779, Training Accuracy= 0.2

Step 416, Minibatch Loss= 2.0024, Training Accuracy= 0.383
Step 417, Minibatch Loss= 1.9088, Training Accuracy= 0.414
Step 418, Minibatch Loss= 1.9505, Training Accuracy= 0.320
Step 419, Minibatch Loss= 1.9282, Training Accuracy= 0.414
Step 420, Minibatch Loss= 1.9884, Training Accuracy= 0.344
Step 421, Minibatch Loss= 1.9274, Training Accuracy= 0.398
Step 422, Minibatch Loss= 1.9408, Training Accuracy= 0.375
Step 423, Minibatch Loss= 1.8975, Training Accuracy= 0.391
Step 424, Minibatch Loss= 1.9818, Training Accuracy= 0.352
Step 425, Minibatch Loss= 1.9007, Training Accuracy= 0.406
Step 426, Minibatch Loss= 1.9363, Training Accuracy= 0.406
Step 427, Minibatch Loss= 1.8946, Training Accuracy= 0.461
Step 428, Minibatch Loss= 1.9502, Training Accuracy= 0.375
Step 429, Minibatch Loss= 1.9414, Training Accuracy= 0.391
Step 430, Minibatch Loss= 1.9279, Training Accuracy= 0.406
Step 431, Minibatch Loss= 2.0062, Training Accuracy= 0.398
Step 432, Minibatch Loss= 1.8573, Training Accuracy= 0.4

Step 556, Minibatch Loss= 1.7934, Training Accuracy= 0.414
Step 557, Minibatch Loss= 1.8680, Training Accuracy= 0.445
Step 558, Minibatch Loss= 1.8388, Training Accuracy= 0.414
Step 559, Minibatch Loss= 1.8439, Training Accuracy= 0.430
Step 560, Minibatch Loss= 1.8355, Training Accuracy= 0.438
Step 561, Minibatch Loss= 1.8187, Training Accuracy= 0.508
Step 562, Minibatch Loss= 1.8759, Training Accuracy= 0.328
Step 563, Minibatch Loss= 1.7872, Training Accuracy= 0.492
Step 564, Minibatch Loss= 1.8987, Training Accuracy= 0.375
Step 565, Minibatch Loss= 1.7882, Training Accuracy= 0.484
Step 566, Minibatch Loss= 1.8325, Training Accuracy= 0.438
Step 567, Minibatch Loss= 1.8332, Training Accuracy= 0.422
Step 568, Minibatch Loss= 1.7934, Training Accuracy= 0.391
Step 569, Minibatch Loss= 1.8424, Training Accuracy= 0.391
Step 570, Minibatch Loss= 1.8498, Training Accuracy= 0.469
Step 571, Minibatch Loss= 1.7909, Training Accuracy= 0.430
Step 572, Minibatch Loss= 1.9241, Training Accuracy= 0.3

...

Step 19372, Minibatch Loss= 0.1570, Training Accuracy= 0.938
Step 19373, Minibatch Loss= 0.2368, Training Accuracy= 0.898
Step 19374, Minibatch Loss= 0.2683, Training Accuracy= 0.914
Step 19375, Minibatch Loss= 0.1977, Training Accuracy= 0.930
Step 19376, Minibatch Loss= 0.1769, Training Accuracy= 0.961
Step 19377, Minibatch Loss= 0.1740, Training Accuracy= 0.953
Step 19378, Minibatch Loss= 0.1724, Training Accuracy= 0.945
Step 19379, Minibatch Loss= 0.2453, Training Accuracy= 0.930
Step 19380, Minibatch Loss= 0.1928, Training Accuracy= 0.914
Step 19381, Minibatch Loss= 0.1353, Training Accuracy= 0.969
Step 19382, Minibatch Loss= 0.1690, Training Accuracy= 0.945
Step 19383, Minibatch Loss= 0.3412, Training Accuracy= 0.875
Step 19384, Minibatch Loss= 0.2546, Training Accuracy= 0.906
Step 19385, Minibatch Loss= 0.1771, Training Accuracy= 0.953
Step 19386, Minibatch Loss= 0.0929, Training Accuracy= 0.977
Step 19387, Minibatch Loss= 0.2995, Training Accuracy= 0.906
Step 19388, Minibatch Lo

Step 19507, Minibatch Loss= 0.3209, Training Accuracy= 0.938
Step 19508, Minibatch Loss= 0.1848, Training Accuracy= 0.938
Step 19509, Minibatch Loss= 0.1689, Training Accuracy= 0.953
Step 19510, Minibatch Loss= 0.1023, Training Accuracy= 0.984
Step 19511, Minibatch Loss= 0.2356, Training Accuracy= 0.906
Step 19512, Minibatch Loss= 0.1779, Training Accuracy= 0.953
Step 19513, Minibatch Loss= 0.1810, Training Accuracy= 0.945
Step 19514, Minibatch Loss= 0.3044, Training Accuracy= 0.891
Step 19515, Minibatch Loss= 0.2201, Training Accuracy= 0.938
Step 19516, Minibatch Loss= 0.2922, Training Accuracy= 0.891
Step 19517, Minibatch Loss= 0.2483, Training Accuracy= 0.922
Step 19518, Minibatch Loss= 0.1232, Training Accuracy= 0.969
Step 19519, Minibatch Loss= 0.1740, Training Accuracy= 0.945
Step 19520, Minibatch Loss= 0.2091, Training Accuracy= 0.930
Step 19521, Minibatch Loss= 0.1563, Training Accuracy= 0.977
Step 19522, Minibatch Loss= 0.1643, Training Accuracy= 0.961
Step 19523, Minibatch Lo

Step 19642, Minibatch Loss= 0.3044, Training Accuracy= 0.906
Step 19643, Minibatch Loss= 0.2317, Training Accuracy= 0.953
Step 19644, Minibatch Loss= 0.2100, Training Accuracy= 0.945
Step 19645, Minibatch Loss= 0.2443, Training Accuracy= 0.922
Step 19646, Minibatch Loss= 0.1168, Training Accuracy= 0.969
Step 19647, Minibatch Loss= 0.1741, Training Accuracy= 0.930
Step 19648, Minibatch Loss= 0.1932, Training Accuracy= 0.930
Step 19649, Minibatch Loss= 0.1765, Training Accuracy= 0.945
Step 19650, Minibatch Loss= 0.1545, Training Accuracy= 0.953
Step 19651, Minibatch Loss= 0.0863, Training Accuracy= 0.977
Step 19652, Minibatch Loss= 0.1525, Training Accuracy= 0.961
Step 19653, Minibatch Loss= 0.2179, Training Accuracy= 0.945
Step 19654, Minibatch Loss= 0.2593, Training Accuracy= 0.945
Step 19655, Minibatch Loss= 0.2229, Training Accuracy= 0.938
Step 19656, Minibatch Loss= 0.1949, Training Accuracy= 0.938
Step 19657, Minibatch Loss= 0.1456, Training Accuracy= 0.953
Step 19658, Minibatch Lo

Step 19777, Minibatch Loss= 0.2954, Training Accuracy= 0.914
Step 19778, Minibatch Loss= 0.2585, Training Accuracy= 0.922
Step 19779, Minibatch Loss= 0.2815, Training Accuracy= 0.922
Step 19780, Minibatch Loss= 0.1541, Training Accuracy= 0.945
Step 19781, Minibatch Loss= 0.2156, Training Accuracy= 0.938
Step 19782, Minibatch Loss= 0.2192, Training Accuracy= 0.945
Step 19783, Minibatch Loss= 0.2435, Training Accuracy= 0.914
Step 19784, Minibatch Loss= 0.1950, Training Accuracy= 0.930
Step 19785, Minibatch Loss= 0.2416, Training Accuracy= 0.914
Step 19786, Minibatch Loss= 0.1506, Training Accuracy= 0.953
Step 19787, Minibatch Loss= 0.2111, Training Accuracy= 0.938
Step 19788, Minibatch Loss= 0.2024, Training Accuracy= 0.938
Step 19789, Minibatch Loss= 0.1627, Training Accuracy= 0.938
Step 19790, Minibatch Loss= 0.1495, Training Accuracy= 0.961
Step 19791, Minibatch Loss= 0.2817, Training Accuracy= 0.930
Step 19792, Minibatch Loss= 0.2092, Training Accuracy= 0.938
Step 19793, Minibatch Lo

Step 19912, Minibatch Loss= 0.1527, Training Accuracy= 0.953
Step 19913, Minibatch Loss= 0.1800, Training Accuracy= 0.961
Step 19914, Minibatch Loss= 0.1627, Training Accuracy= 0.938
Step 19915, Minibatch Loss= 0.1108, Training Accuracy= 0.969
Step 19916, Minibatch Loss= 0.2134, Training Accuracy= 0.914
Step 19917, Minibatch Loss= 0.3536, Training Accuracy= 0.891
Step 19918, Minibatch Loss= 0.1658, Training Accuracy= 0.953
Step 19919, Minibatch Loss= 0.1027, Training Accuracy= 0.961
Step 19920, Minibatch Loss= 0.1604, Training Accuracy= 0.961
Step 19921, Minibatch Loss= 0.1477, Training Accuracy= 0.953
Step 19922, Minibatch Loss= 0.1234, Training Accuracy= 0.961
Step 19923, Minibatch Loss= 0.3510, Training Accuracy= 0.922
Step 19924, Minibatch Loss= 0.2211, Training Accuracy= 0.922
Step 19925, Minibatch Loss= 0.2949, Training Accuracy= 0.914
Step 19926, Minibatch Loss= 0.1628, Training Accuracy= 0.938
Step 19927, Minibatch Loss= 0.1197, Training Accuracy= 0.961
Step 19928, Minibatch Lo